# Entity Explorer - Account
 <details>
     <summary>&nbsp;<u>Details...</u></summary>

 **Notebook Version:** 1.0<br>
 **Python Version:** Python 3.6 (including Python 3.6 - AzureML)<br>
 **Required Packages**: kqlmagic, msticpy, pandas, numpy, matplotlib, networkx, ipywidgets, ipython, dnspython, ipwhois, folium, maxminddb_geolite2<br>
 **Platforms Supported**:
 - Azure Notebooks Free Compute
 - Azure Notebooks DSVM
 - OS Independent

 **Data Sources Required**:
 - Log Analytics - SecurityAlert, SecurityEvent, HuntingBookmark, Syslog, AAD SigninLogs, AzureActivity, OfficeActivity, ThreatIndicator
 - (Optional) - VirusTotal, AlienVault OTX, IBM XForce, Open Page Rank, (all require accounts and API keys)
 </details>

 Brings together a series of queries and visualizations to help you determine the security state of an Account. The account can be a Windows or Linux account or an Azure Active Directory/Office 365 account.

This notebook uses [*MSTIC Notebooklets*](https://msticnb.readthedocs.io/en/latest/) for most of its operations. 
These are collections of logic taken from notebooks and packaged in an easy-to-use form.
The Notebooklet contains sections for reviewing activity for Host accounts
(Linux and Windows) and for Azure Active Directory accounts. Depending on the
account type, it will automatically select the correct queries and
other operations to drill down into the data.

<!DOCTYPE html>
<html>
  <head>
  </head>
  <body>
    <h1>Contents<span class="tocSkip"></span></h1>
    <div class="toc">
      <ul class="toc-item">
        <li><span><a href="#Hunting-Hypothesis">Hunting Hypothesis</a></span></li>
        <li><span><a href="#Notebook-initialization">Notebook initialization</a></span></li>
        <li><span><a href="#Enter-account-name-and-query-time">Enter account
              name and query time</a></span></li>
        <li><span><a href="#Run-the-main-account_summary-function">Run the main
              account_summary function</a></span></li>
        <li><span><a href="#Retrieve-additional-data-for-the-selected-account">Retrieve
              additional data for the selected account</a></span></li>
        <li><span><a href="#Browse-alerts">Browse alerts</a></span></li>
        <li><span><a href="#Browse-other-event-data">Browse other event data</a></span></li>
        <li><span><a href="#Additional-properties-and-methods-of-the-result-object">
              Additional properties and methods of the result object<br>
            </a></span></li>
            <ul class="toc-item">
            <li><span><a href="#IP-Summary-table">IP Summary table</a></span></li>
            <li><span><a href="#IP-Geo-Map">IP Geo-Map</a></span></li>
            </ul>
        <li><span><a href="#Viewing-the-Result-class">Viewing the Result class</a></span></li>
        <li><span><a href="#Use-other-notebooklets-and-pivots-functions-to-drill-down-on-other-entities">
              Use other notebooklets and pivots functions to drill down on other
              entities</a></span></li>
        <li><span><a href="#More-information">More information</a></span></li>
      </ul>
    </div>
    
  </body>
</html>


## Hunting Hypothesis
Our broad initial hunting hypothesis is that a we have received account name entity which is suspected to be compromised and is being used malicious manner in internal networks, we will need to hunt from a range of different positions to validate or disprove this hypothesis.

---
# Notebook initialization
This should complete without errors. If you encounter errors or warnings look at the following notebooks:

- <a href="https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb">Getting Started Notebook</a>
- [TroubleShootingNotebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/TroubleShootingNotebooks.ipynb)
- [ConfiguringNotebookEnvironment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)

<details>
    <summary>&nbsp;<u>Details...</u></summary>
The next cell:
- Checks for the correct Python version
- Checks versions and optionally installs required packages
- Imports the required packages into the notebook
- Sets a number of configuration options.

If you are running in the Azure Sentinel Notebooks environment (Azure Notebooks or Azure ML) you can run live versions of these notebooks:
- [Getting Started](./A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb)
- [Run TroubleShootingNotebooks](./TroubleShootingNotebooks.ipynb)
- [Run ConfiguringNotebookEnvironment](./ConfiguringNotebookEnvironment.ipynb)

You may also need to do some additional configuration to successfully use functions such as Threat Intelligence service lookup and Geo IP lookup. 
There are more details about this in the `ConfiguringNotebookEnvironment` notebook and in these documents:
- [msticpy configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [Threat intelligence provider configuration](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html#configuration-file)
</details>

In [25]:
from pathlib import Path
from IPython.display import display, HTML

REQ_PYTHON_VER=(3, 6)
REQ_MSTICPY_VER=(1, 0, 0)

display(HTML("<h3>Starting Notebook setup...</h3>"))
if Path("./utils/nb_check.py").is_file():
    from utils.nb_check import check_versions
    check_versions(REQ_PYTHON_VER, REQ_MSTICPY_VER)

# You may need to manually install msticpy with
# !pip install msticpy[azsentinel]

from msticpy.nbtools import nbinit
nbinit.init_notebook(
    namespace=globals(),
    additional_packages=["msticnb>=0.2"],
);


msticpy version installed: 1.0.0rc4 latest published: 0.9.0
Latest version is installed.
Processing imports....
All packages are already installed
Checking configuration....
No errors found.
No warnings found.
Exception while checking configuration:
cannot unpack non-iterable NoneType object
Setting notebook options....


### Get Workspace and Authenticate
<details>
    <summary>&nbsp;<u>Details...</u></summary>
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.<br>
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. <br>
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. <br>
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.<br>

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

Note: you may occasionally see a JavaScript warning displayed at the end of the authentication - you can safely ignore this.<br>
On successful authentication you should see a ```popup schema``` button.
To find your Workspace Id go to [Log Analytics](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.
</details>

In [26]:
# Authentication
# ws_config = WorkspaceConfig()
qry_prov = QueryProvider(data_environment="LogAnalytics")
qry_prov.connect(WorkspaceConfig("CyberSecuritySoc"))

#### Authentication and Configuration problems?

If you are having problems, expand the details section below
<br>
<details>
    <summary>Click for details about configuring your authentication parameters</summary>
    
    
The notebook is expecting your Azure Sentinel Tenant ID and Workspace ID to be configured in one of the following places:
- `config.json` in the current folder
- `msticpyconfig.yaml` in the current folder or location specified by `MSTICPYCONFIG` environment variable.
    
For help with setting up your `config.json` file (if this hasn't been done automatically) see the [`ConfiguringNotebookEnvironment`](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb) notebook in the root folder of your Azure-Sentinel-Notebooks project. This shows you how to obtain your Workspace and Subscription IDs from the Azure Sentinel Portal. You can use the SubscriptionID to find your Tenant ID). To view the current `config.json` run the following in a code cell.

```%pfile config.json```

For help with setting up your `msticpyconfig.yaml` see the [Setup](#Setup) section at the end of this notebook and the [ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)
</details>

## Import notebooklets and initialize pivot functions

These are needed for many of the notebook operations

In [27]:
import msticnb as nb
pivot = Pivot(globals())
nb.init(query_provider=qry_prov)

Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank
Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank
Notebooklets: Loaded providers: AzureSentinel, geolitelookup, tilookup


# Enter account name and query time window
Type the account name that you want to search for and the time bounds over which you want to search. 

You can specify the account as:

- a simple user name (e.g. `alice`)
- a user principal name (`alice@contoso.com`)
- a qualified windows user name `mydomain\alice`

In the second two cases the domain qualifier will be stripped off before the search. The search is not case sensitive and will match full substrings. E.g. `bob` will match `domain\bob` and `bob@contoso.com` but not `bobg` or `bo`.

In [28]:
account_txt = nbwidgets.GetText(prompt='Enter the Account name to search for:')
md("<hr>")
display(account_txt)
md("<hr>")
pivot.edit_query_time()

Text(value='seb', description='Enter the Account name to search for:', layout=Layout(width='50%'), style=Descr…

---

# Run the main *account_summary* function

If more than one matching account name is found, all matches will be shown. 

You can select each one to view more details about the account.

> Note: this is run as a pivot function from the Account entity.
> You can also run it directly as a Notebooklet function by 
> creating and instance of `nb.nblt.azsent.account.AccountSummary()`
> and executing the `run()` method

```python
>>> acct_nb = nb.nblt.azsent.account.AccountSummary()
>>> acct_result = acct_nb.run(pivot.timespan, "account_name")  # subtitute the "account_name" with the name to search
```

In [29]:
Account = entities.Account
acc_result = Account.nblt.account_summary(value=account_txt.value)

Loading BokehJS ...

Account(Name=seb@seccxp.ninja, UPNSuffix=seccxp.ninja, AadTenantId=4b2462a4-bbee-495a-a0...)

,TimeGenerated,TenantId,SourceSystem,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationDetails,AuthenticationMethodsUsed,AuthenticationProcessingDetails,AuthenticationRequirement,...,RiskLevelAggregated,RiskLevelDuringSignIn,RiskState,ResourceDisplayName,ResourceIdentity,ServicePrincipalId,ServicePrincipalName,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,UserType,FlaggedForReview,IPAddressFromResourceProvider,SignInIdentifier,SignInIdentifierType,ResourceTenantId,HomeTenantId,Type,AccountName,Source
0,2021-04-01 13:59:12.319000+00:00,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,9b481072-380c-42e7-867f-ea639bf80155,Microsoft.aadiam,Microsoft.aadiam,,Admin,4,BE,,Microsoft Teams Web Client,5e3ce6c0-2b1f-4285-8d4b-75ee78787346,"[\r\n {\r\n ""authenticationStepDateTime"": ""2021-04-01T13:59:12.3194749+00:00"",\r\n ""authe...",,"[\r\n {\r\n ""key"": ""Login Hint Present"",\r\n ""value"": ""True""\r\n },\r\n {\r\n ""key"":...",singleFactorAuthentication,...,none,none,none,Office 365 Exchange Online,00000002-0000-0ff1-ce00-000000000000,,,{'errorCode': 0},,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.438...",Admin,bdb72692-2275-412b-8af0-ea21c2ebd174,seb@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,seb@seccxp.ninja,AzureActiveDirectory


---

# Retrieve additional data for the selected account

The result returned from the last cell has a number of properties and methods that you can use
to retrieve and view further information.

The main one for this notebooklet is `get_additional_data`.

In [30]:
acc_result.get_additional_data()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

OperationCount  \
UserPrincipalName Type           IPAddress                AppResourceProvider                          UserType                   
seb@seccxp.ninja  OfficeActivity                          MicrosoftTeams                               Regular               12   
                                 104.209.247.23           Exchange                                     Regular                1   
                                 157.56.162.53            SharePoint                                   Regular               25   
                                 2603:10b6:5:1c0:cafe::5f Exchange                                     Regular                1   
                                 2603:10b6:5:9::18        Exchange                                     Regular                1   
                                 40.83.137.154            OneDrive                                     Regular               39   
                                 40.91.208.65             Exchange                                     Regular               25   
                  SigninLogs     109.88.218.99            Azure Advanced Threat Protection             Member                 4   
                                                          Azure Portal                                 Member                 2   
                                                          Microsoft 365 Security and Compliance Center Member                 3   
                                                          Microsoft Cloud App Security                 Member                 1   
                                                          Microsoft Teams Web Client                   Member                68   
                                                          Office 365 Exchange Online                   Member                 1   
                                                          Office365 Shell WCSS-Client                  Member                11   

                                                                                                                               OperationTypes  \
UserPrincipalName Type           IPAddress                AppResourceProvider                          UserType                                 
seb@seccxp.ninja  OfficeActivity                          MicrosoftTeams                               Regular          [TeamsSessionStarted]   
                                 104.209.247.23           Exchange                                     Regular                         [Send]   
                                 157.56.162.53            SharePoint                                   Regular                 [FileAccessed]   
                                 2603:10b6:5:1c0:cafe::5f Exchange                                     Regular            [MailItemsAccessed]   
                                 2603:10b6:5:9::18        Exchange                                     Regular            [MailItemsAccessed]   
                                 40.83.137.154            OneDrive                                     Regular   [FileAccessed, FileModified]   
                                 40.91.208.65             Exchange                                     Regular            [MailItemsAccessed]   
                  SigninLogs     109.88.218.99            Azure Advanced Threat Protection             Member              [Sign-in activity]   
                                                          Azure Portal                                 Member              [Sign-in activity]   
                                                          Microsoft 365 Security and Compliance Center Member              [Sign-in activity]   
                                                          Microsoft Cloud App Security                 Member              [Sign-in activity]   
                                                          Microsoft Teams Web Client                   Member              [Sign-in

nan does not appear to be an IPv4 or IPv6 address


Count  \
Source               Operation         UserAgent                                                                                            IpAddress                AsnDescription                  CountryCode          
AzureActiveDirectory Sign-in activity  Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0... 109.88.218.99            ASBRUTELE VOO, BE               BE               5   
                                       Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389... 109.88.218.99            ASBRUTELE VOO, BE               BE              85   
Office365            FileAccessed                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US              14   
                                       azure-logic-apps/1.0 (workflow 5cd83b3b4b1b4e77afa122aafdad7fdc; version 08585848427717773653) mi... 157.56.162.53            MICROSOFT-CORP-MSN-AS-BLOCK, US US              25   
                     FileModified                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US              25   
                     MailItemsAccessed                                                                                                      2603:10b6:5:1c0:cafe::5f NA                              US               1   
                                                                                                                                            2603:10b6:5:9::18        NA                              US               1   
                                                                                                                                            40.91.208.65             MICROSOFT-CORP-MSN-AS-BLOCK, US NL              25   
                     Send                                                                                                                   104.209.247.23           MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   

                                                                                                                                                                                                                                   FirstOperation  \
Source               Operation         UserAgent                                                                                            IpAddress                AsnDescription                  CountryCode                                    
AzureActiveDirectory Sign-in activity  Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0... 109.88.218.99            ASBRUTELE VOO, BE               BE          2021-04-01 13:49:08.481000+00:00   
                                       Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389... 109.88.218.99            ASBRUTELE VOO, BE               BE          2021-03-31 16:04:38.185000+00:00   
Office365            FileAccessed                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US                 2021-04-01 15:44:12+00:00   
                                       azure-logic-apps/1.0 (workflow 5cd83b3b4b1b4e77afa122aafdad7fdc; version 08585848427717773653) mi... 157.56.162.53            MICROSOFT-CORP-MSN-AS-BLOCK, US US                 2021-04-01 15:44:10+00:00   
                     FileModified                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US                 2021-04-01 15:44:14+00:00   
                     MailItemsAccessed                                               

---

# Browse alerts
If there are any alerts referencing this account name they can be viewed
by calling the `acc_result.browse_alerts()` function.

In [23]:
acc_result.browse_alerts()

,0
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
TimeGenerated,2021-03-28 09:15:12.881000+00:00
AlertDisplayName,Azure Security Center unusual storage access
AlertName,Azure Security Center unusual storage access
Severity,Medium
Description,Azure Security Center unusual storage access
ProviderName,ASI Scheduled Alerts
VendorName,Microsoft
VendorOriginalId,9ff35ff9-f4bc-4341-abd9-0e91735a790f
SystemAlertId,5ff9cdd5-4185-2f39-0f5f-545b6d4fd725


---

# Browse other event data

You can use a simple view to make it easier to see the details of individual events
by calling the "view_events" method.

You need to supply the name of the result attribute that you want to view plus
one or more summary columns (as a list of strings).

In [24]:
acc_result.view_events(attrib="azure_activity", summary_cols=["SourceSystem", "Operation", "IPAddress"])

,305
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
SourceSystem,OfficeActivityManager
TimeGenerated,2021-03-27 15:44:25+00:00
ResourceId,https://seccxpninja.sharepoint.com/sites/CxEData/Shared Documents/Project Delta/Product Marketin...
Operation,FileAccessed
AppResourceProvider,SharePoint
IPAddress,157.56.162.53
UserAgent,azure-logic-apps/1.0 (workflow 5cd83b3b4b1b4e77afa122aafdad7fdc; version 08585848427717773653) m...
UserId,seb@seccxp.ninja
UserPrincipalName,seb@seccxp.ninja


## Browse events with alternative sorting
You can pass a DatFrame to view events instead of an attribute name.

This means that you can apply sorting or filtering of the data before viewing it.
Here is an example sorting by IPAddress.

In [ ]:
acc_result.view_events(
    data=acc_result.azure_activity.sort_values("IPAddress"),
    summary_cols=["SourceSystem", "Operation", "IPAddress"]
)

---

# Additional properties and methods of the result object

These are static properties - usually DataFrames or visualizations. 
You can access each of these to see or manipulate the retrieved data.

To see help on the available attributes type:
```python
>>> help(acc_result)
```
To see the available methods type:
```python
>>> acc_result.list_methods()
```
> Note, for the AccountSummary notebooklet, the two main data retrieval methods are:<br>
> - run<br>
> - get_additional_data<br>
> There are several other methods that allow you to view individual plots
> or subsets of the data (such as alerts).

To view help on a specific method type:
```python
>>> help(acc_result.method_name())
```

In [ ]:
acc_result.display_alert_timeline()

## IP Summary table

In [31]:
acc_result.ip_summary

Count  \
Source               Operation         UserAgent                                                                                            IpAddress                AsnDescription                  CountryCode          
AzureActiveDirectory Sign-in activity  Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0... 109.88.218.99            ASBRUTELE VOO, BE               BE               5   
                                       Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389... 109.88.218.99            ASBRUTELE VOO, BE               BE              85   
Office365            FileAccessed                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US              14   
                                       azure-logic-apps/1.0 (workflow 5cd83b3b4b1b4e77afa122aafdad7fdc; version 08585848427717773653) mi... 157.56.162.53            MICROSOFT-CORP-MSN-AS-BLOCK, US US              25   
                     FileModified                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US              25   
                     MailItemsAccessed                                                                                                      2603:10b6:5:1c0:cafe::5f NA                              US               1   
                                                                                                                                            2603:10b6:5:9::18        NA                              US               1   
                                                                                                                                            40.91.208.65             MICROSOFT-CORP-MSN-AS-BLOCK, US NL              25   
                     Send                                                                                                                   104.209.247.23           MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   

                                                                                                                                                                                                                                   FirstOperation  \
Source               Operation         UserAgent                                                                                            IpAddress                AsnDescription                  CountryCode                                    
AzureActiveDirectory Sign-in activity  Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0... 109.88.218.99            ASBRUTELE VOO, BE               BE          2021-04-01 13:49:08.481000+00:00   
                                       Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389... 109.88.218.99            ASBRUTELE VOO, BE               BE          2021-03-31 16:04:38.185000+00:00   
Office365            FileAccessed                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US                 2021-04-01 15:44:12+00:00   
                                       azure-logic-apps/1.0 (workflow 5cd83b3b4b1b4e77afa122aafdad7fdc; version 08585848427717773653) mi... 157.56.162.53            MICROSOFT-CORP-MSN-AS-BLOCK, US US                 2021-04-01 15:44:10+00:00   
                     FileModified                                                                                                           40.83.137.154            MICROSOFT-CORP-MSN-AS-BLOCK, US US                 2021-04-01 15:44:14+00:00   
                     MailItemsAccessed                                               

## IP Geo-Map
> **Note**: you may need to zoom out to see the data.

In [32]:
acc_result.get_geoip_map()

---

# Viewing the Result class
You can view all of the data in the results class by "running" it in a cell

> Note: due to the way Jupyter display Javascript objects the plots may
> appear out of order.

Most of the properties of the results class are pandas DataFrames - 
you can use these directly for further analysis. Other property types
include entities and visualizations.

The DataFrames displayed by running the result object are truncated
to the first five rows.

In [ ]:
acc_result

---

# Use other notebooklets and pivots functions to drill down on other entities

You may want to drill down on other entities in the Account data.
You can use methods of the IpAddress or Host entities, for example,
to look at these in more detail.

In [33]:
IpAddress = entities.IpAddress
ip_result = IpAddress.nblt.ip_address_summary("40.91.208.65")

,nir,asn_registry,asn,asn_cidr,asn_country_code,asn_date,asn_description,query,nets,raw,referral,raw_referral
0,None,arin,8075,40.80.0.0/12,US,2015-02-23,"MICROSOFT-CORP-MSN-AS-BLOCK, US",40.91.208.65,"{'cidr': '40.125.0.0/17, 40.124.0.0/16, 40.74.0.0/15, 40.96.0.0/12, 40.112.0.0/13, 40.80.0.0/12,...",None,None,None


GeoLocation(CountryCode=NL, CountryName=Netherlands, State=North Holland, City=Amsterdam, Lo...)

,Ioc,IocType,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
OTX,40.91.208.65,ipv4,None,OTX,True,information,"{'pulse_count': 0, 'sections_available': ['general', 'geo', 'reputation', 'url_list', 'passive_d...","{'whois': 'http://whois.domaintools.com/40.91.208.65', 'reputation': 0, 'indicator': '40.91.208....",https://otx.alienvault.com/api/v1/indicators/IPv4/40.91.208.65/general,0
OPR,40.91.208.65,ipv4,None,OPR,False,information,IoC type ipv4 not supported.,None,None,1
Tor,40.91.208.65,ipv4,None,Tor,True,information,Not found.,None,https://check.torproject.org/exit-addresses,0
VirusTotal,40.91.208.65,ipv4,None,VirusTotal,True,information,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'detected_urls': [], 'positives': 0...","{'asn': 8075, 'undetected_urls': [['http://40.91.208.65/', '8ad4f4804ca454981931832fa2652d82ec97...",https://www.virustotal.com/vtapi/v2/ip-address/report,0
XForce,40.91.208.65,ipv4,None,XForce,True,warning,"{'score': 1, 'cats': {}, 'categoryDescriptions': {}, 'reason': 'Regional Internet Registry', 're...","{'ip': '40.91.208.65', 'history': [{'created': '2012-03-22T07:26:00.000Z', 'reason': 'Regional I...",https://api.xforce.ibmcloud.com/ipr/40.91.208.65,0


In [ ]:
ip_result.browse_ti_results()

---

# More information:

## Notebooklets and Pivots
[Notebooklets](https://msticnb.readthedocs.io/en/latest/)

[Pivot functions](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

## Notebook/MSTICPy configuration
[MSTICPy Configuration guide](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)

[ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)